# IV. FORECASTING MODEL TUNING

We re-evaluate our selections of machine learning on our new data from April 2022 to June 2023 that contains both cloud mask and overview RGB images from the Himawari- 8 satellite. 

- Section IV-A covers the selection of model candidates and features.

- The input for this notebook is `dataset_realistic_cm_r_5x5.csv`, which contains data on cloud extraction and clear-sky irradiance for the specified date, time, and site.
- The outputs of this notebook are the estimated irradiance values from each model, including three models:
    - Extra Trees model: `Ihat_et_test_set.csv`
    - LightGBM model: `Ihat_lightgbm_v2_test_set.csv`
    - Random Forest model: `Ihat_rf_test_set.csv`

# Preprocess data

#### You can scrip this section if you have file `train_df.csv` and `train_df.csv`

In [23]:
# import data 
import pandas as pd

df = pd.read_csv('dataset_realistic_cm_r_5x5.csv', parse_dates=['Datetime'])
df['site_name'] = df['site_name'].astype(str).str.zfill(2)
df['date'] = df['Datetime'].dt.date.astype(str)
df['date'] = df['date'] +" "+ df['site_name']
df['k'] = df['I'] / df['Iclr']
df.set_index('Datetime', inplace=True)
df

,site_name,I,Iclr,latt,long,CI,CI_1,CI_2,CI_3,CI_4,...,R_18,R_19,R_20,R_21,R_22,R_23,R_24,R_25,date,k
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-04-01 07:30:00+07:00,01,235.57840,265.993700,14.00523,100.519403,14.0,20.0,15.0,15.0,20.0,...,82.0,76.0,77.0,94.0,81.0,79.0,75.0,76.0,2022-04-01 01,0.885654
2022-04-01 07:45:00+07:00,01,322.49360,334.819476,14.00523,100.519403,14.0,20.0,15.0,15.0,20.0,...,82.0,76.0,77.0,94.0,81.0,79.0,75.0,76.0,2022-04-01 01,0.963187
2022-04-01 08:00:00+07:00,01,357.68410,402.581345,14.00523,100.519403,13.0,12.0,15.0,17.0,21.0,...,75.0,76.0,78.0,79.0,76.0,76.0,75.0,77.0,2022-04-01 01,0.888477
2022-04-01 08:15:00+07:00,01,413.50590,468.732095,14.00523,100.519403,13.0,12.0,15.0,17.0,21.0,...,75.0,76.0,78.0,79.0,76.0,76.0,75.0,77.0,2022-04-01 01,0.882180
2022-04-01 08:30:00+07:00,01,471.19330,532.843024,14.00523,100.519403,13.0,12.0,15.0,17.0,21.0,...,75.0,76.0,78.0,79.0,76.0,76.0,75.0,77.0,2022-04-01 01,0.884300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 16:00:00+07:00,56,91.85729,666.692427,13.50130,100.135400,211.0,218.0,230.0,229.0,229.0,...,153.0,153.0,153.0,153.0,147.0,151.0,156.0,158.0,2023-06-30 56,0.137781
2023-06-30 16:15:00+07:00,56,107.00000,609.298561,13.50130,100.135400,211.0,218.0,230.0,229.0,229.0,...,153.0,153.0,153.0,153.0,147.0,151.0,156.0,158.0,2023-06-30 56,0.175612
2023-06-30 16:30:00+07:00,56,113.72660,549.564992,13.50130,100.135400,211.0,218.0,230.0,229.0,229.0,...,153.0,153.0,153.0,153.0,147.0,151.0,156.0,158.0,2023-06-30 56,0.206939


In [24]:
# create new features and encode them

df = df.between_time('07:00', '17:00')
df['hour'] = df.index.hour
df['hour_encode1'] = df['hour'] - 11.5
df['hour_encode2'] = (df['hour'] - 7)/(17-7)
df['day'] = df.index.dayofyear
df['month'] = df.index.month
df.reset_index(inplace=True)
df

,Datetime,site_name,I,Iclr,latt,long,CI,CI_1,CI_2,CI_3,...,R_23,R_24,R_25,date,k,hour,hour_encode1,hour_encode2,day,month
0,2022-04-01 07:30:00+07:00,01,235.57840,265.993700,14.00523,100.519403,14.0,20.0,15.0,15.0,...,79.0,75.0,76.0,2022-04-01 01,0.885654,7,-4.5,0.0,91,4
1,2022-04-01 07:45:00+07:00,01,322.49360,334.819476,14.00523,100.519403,14.0,20.0,15.0,15.0,...,79.0,75.0,76.0,2022-04-01 01,0.963187,7,-4.5,0.0,91,4
2,2022-04-01 08:00:00+07:00,01,357.68410,402.581345,14.00523,100.519403,13.0,12.0,15.0,17.0,...,76.0,75.0,77.0,2022-04-01 01,0.888477,8,-3.5,0.1,91,4
3,2022-04-01 08:15:00+07:00,01,413.50590,468.732095,14.00523,100.519403,13.0,12.0,15.0,17.0,...,76.0,75.0,77.0,2022-04-01 01,0.882180,8,-3.5,0.1,91,4
4,2022-04-01 08:30:00+07:00,01,471.19330,532.843024,14.00523,100.519403,13.0,12.0,15.0,17.0,...,76.0,75.0,77.0,2022-04-01 01,0.884300,8,-3.5,0.1,91,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867808,2023-06-30 16:00:00+07:00,56,91.85729,666.692427,13.50130,100.135400,211.0,218.0,230.0,229.0,...,151.0,156.0,158.0,2023-06-30 56,0.137781,16,4.5,0.9,181,6
867809,2023-06-30 16:15:00+07:00,56,107.00000,609.298561,13.50130,100.135400,211.0,218.0,230.0,229.0,...,151.0,156.0,158.0,2023-06-30 56,0.175612,16,4.5,0.9,181,6
867810,2023-06-30 16:30:00+07:00,56,113.72660,549.564992,13.50130,100.135400,211.0,218.0,230.0,229.0,...,151.0,156.0,158.0,2023-06-30 56,0.206939,16,4.5,0.9,181,6
867811,2023-06-30 16:45:00+07:00,56,108.15760,487.772006,13.50130,100.135400,211.0,218.0,230.0,229.0,...,151.0,156.0,158.0,2023-06-30 56,0.221738,16,4.5,0.9,181,6


In [25]:
import datetime
import numpy as np

# We will use the sine and cosine transformations of the timestamp to encode the cyclical nature of the time of day.
timestamp_s = pd.to_datetime(df['Datetime']).map(datetime.datetime.timestamp)

# The timestamp is in seconds, so we must calculate the number of seconds in a day before applying the sine transformation.
day = 24*60*60 
df['day_sin'] = (np.sin(timestamp_s * (2*np.pi/day))).values
df['day_cos'] = (np.cos(timestamp_s * (2*np.pi/day))).values
df

,Datetime,site_name,I,Iclr,latt,long,CI,CI_1,CI_2,CI_3,...,R_25,date,k,hour,hour_encode1,hour_encode2,day,month,day_sin,day_cos
0,2022-04-01 07:30:00+07:00,01,235.57840,265.993700,14.00523,100.519403,14.0,20.0,15.0,15.0,...,76.0,2022-04-01 01,0.885654,7,-4.5,0.0,91,4,0.130526,0.991445
1,2022-04-01 07:45:00+07:00,01,322.49360,334.819476,14.00523,100.519403,14.0,20.0,15.0,15.0,...,76.0,2022-04-01 01,0.963187,7,-4.5,0.0,91,4,0.195090,0.980785
2,2022-04-01 08:00:00+07:00,01,357.68410,402.581345,14.00523,100.519403,13.0,12.0,15.0,17.0,...,77.0,2022-04-01 01,0.888477,8,-3.5,0.1,91,4,0.258819,0.965926
3,2022-04-01 08:15:00+07:00,01,413.50590,468.732095,14.00523,100.519403,13.0,12.0,15.0,17.0,...,77.0,2022-04-01 01,0.882180,8,-3.5,0.1,91,4,0.321439,0.946930
4,2022-04-01 08:30:00+07:00,01,471.19330,532.843024,14.00523,100.519403,13.0,12.0,15.0,17.0,...,77.0,2022-04-01 01,0.884300,8,-3.5,0.1,91,4,0.382683,0.923880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867808,2023-06-30 16:00:00+07:00,56,91.85729,666.692427,13.50130,100.135400,211.0,218.0,230.0,229.0,...,158.0,2023-06-30 56,0.137781,16,4.5,0.9,181,6,0.707107,-0.707107
867809,2023-06-30 16:15:00+07:00,56,107.00000,609.298561,13.50130,100.135400,211.0,218.0,230.0,229.0,...,158.0,2023-06-30 56,0.175612,16,4.5,0.9,181,6,0.659346,-0.751840
867810,2023-06-30 16:30:00+07:00,56,113.72660,549.564992,13.50130,100.135400,211.0,218.0,230.0,229.0,...,158.0,2023-06-30 56,0.206939,16,4.5,0.9,181,6,0.608761,-0.793353
867811,2023-06-30 16:45:00+07:00,56,108.15760,487.772006,13.50130,100.135400,211.0,218.0,230.0,229.0,...,158.0,2023-06-30 56,0.221738,16,4.5,0.9,181,6,0.555570,-0.831470


In [ ]:
# split the data into train and test sets
from utils import SolarSplitter

train_df, test_df, train_info_df, test_info_df, cluster_info_df = SolarSplitter.split_train_test_sky_condition(df, train_frac=0.6, random_state=42)

# Train model

In [28]:
# load the train and test sets that we have saved
import pandas as pd
train_df = pd.read_csv('/Users/nuttamon/Documents/project/solar_map/data/train_df.csv')
test_df = pd.read_csv('/Users/nuttamon/Documents/project/solar_map/data/test_df.csv')

In [42]:
train = train_df[['Iclr', 'CI', 'R', 'hour_encode1', 'latt' , 'day', 'month', 'long', 'I']]
train

,Iclr,CI,R,hour_encode1,latt,day,month,long,I
0,265.9937,14.0000,78.0000,-4.5000,14.0052,91,4,100.5194,235.5784
1,334.8195,14.0000,78.0000,-4.5000,14.0052,91,4,100.5194,322.4936
2,402.5813,13.0000,74.0000,-3.5000,14.0052,91,4,100.5194,357.6841
3,468.7321,13.0000,74.0000,-3.5000,14.0052,91,4,100.5194,413.5059
4,532.8430,13.0000,74.0000,-3.5000,14.0052,91,4,100.5194,471.1933
...,...,...,...,...,...,...,...,...,...
520685,666.6924,211.0000,158.0000,4.5000,13.5013,181,6,100.1354,91.8573
520686,609.2986,211.0000,158.0000,4.5000,13.5013,181,6,100.1354,107.0000
520687,549.5650,211.0000,158.0000,4.5000,13.5013,181,6,100.1354,113.7266
520688,487.7720,211.0000,158.0000,4.5000,13.5013,181,6,100.1354,108.1576


## Train AutoML

In [17]:
import pycaret
from pycaret.regression import *


# Set display results
pd.options.display.float_format = '{:,.4f}'.format
%config InlineBackend.figure_format = 'retina'

# PyCaret version
print(f'PyCaret version: {pycaret.__version__}')

PyCaret version: 3.3.1


In [43]:
# Set up the PyCaret environment
regression = setup(
    data=train, 
    target='I',  # Specify 'I' as the target column
    train_size=0.8,
    numeric_features=['Iclr', 'CI', 'R', 'hour_encode1', 'latt' ,'long'],
    numeric_imputation='mean',  # For deployment purposes
    normalize=True,
    normalize_method='minmax',
    polynomial_features=True,
    polynomial_degree=2,
    use_gpu=False,
    fold=5,
    n_jobs=-1,
)

,Description,Value
0,Session id,3148
1,Target,I
2,Target type,Regression
3,Original data shape,"(520690, 9)"
4,Transformed data shape,"(520690, 45)"
5,Transformed train set shape,"(416552, 45)"
6,Transformed test set shape,"(104138, 45)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


In [22]:
# Compare models
estimatedModels = compare_models(sort = 'MAE', n_select = 10)

In [ ]:
# Print the estimated models
print(estimatedModels)

In [ ]:
# Plot the estimated models
plot_model(estimatedModels[0], plot = 'feature')

In [ ]:
# Create a model
model = create_model('rf')

In [ ]:
# Tune the model 

custom_grid = {
    'max_depth': [None, 5,10,20],  
    'max_features': [1,4,6,12],  
    'min_samples_leaf': [1,10,20,50]
}

# Tune the model with the custom grid
tuned_rf = tune_model(model, custom_grid=custom_grid)

In [ ]:
# Print the tuned model
tuned_rf.get_params()

In [ ]:
# Plot the tuned model
finalModel_rf = finalize_model(tuned_rf)

In [ ]:
# save model
save_model(finalModel_rf, '/mode_tuned/model_rf')

# Load model

In [40]:
loaded_model = load_model('model/model_lightgbm')
loaded_model

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/var/folders/br/dcj9wlx564d4m718qfgt5pq00000gn/T/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Iclr', 'CI', 'R', 'hour_encode1',
                                             'latt', 'long'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('polynomial_features',
                 TransformerWrapper(transformer=PolynomialFeatures(include_bias=False))),
                ('normalize', TransformerWrapper(transformer=MinMaxScaler())),
                ('actual_estimator',
                 LGBMRegressor(min_child_samples=100, n_jobs=-1, num_leaves=200,
                               random_state=8973))])

In [41]:

# Save results
  
test = test_df[['Iclr', 'CI', 'R', 'hour_encode1','latt' ,'long', 'day', 'month']].copy()
test_df['Datetime'] = pd.to_datetime(test_df['Datetime'])

test['I_hat'] = loaded_model.predict(test[['Iclr', 'CI', 'R', 'hour_encode1','latt' ,'long', 'day', 'month']])
test['site_name'] = test_df['site_name']
test['Datetime'] = test_df.Datetime
test['I'] = test_df.I
test.set_index('Datetime', inplace=True)
test['Date'] = test.index.date.astype(str)
test['hour'] = test.index.hour
test


,Iclr,CI,R,hour_encode1,latt,long,day,month,I_hat,site_name,I,Date,hour
Datetime,,,,,,,,,,,,,
2022-04-05 07:00:00+07:00,141.2417,23.0000,52.0000,-4.5000,14.0052,100.5194,95,4,132.9742,ISL001,183.5975,2022-04-05,7
2022-04-05 07:15:00+07:00,209.5181,23.0000,52.0000,-4.5000,14.0052,100.5194,95,4,176.8404,ISL001,240.5533,2022-04-05,7
2022-04-05 07:30:00+07:00,278.6117,36.0000,76.0000,-4.5000,14.0052,100.5194,95,4,213.1422,ISL001,164.2464,2022-04-05,7
2022-04-05 07:45:00+07:00,347.2928,36.0000,76.0000,-4.5000,14.0052,100.5194,95,4,282.4012,ISL001,176.0619,2022-04-05,7
2022-04-05 08:00:00+07:00,414.8302,44.0000,147.0000,-3.5000,14.0052,100.5194,95,4,241.0597,ISL001,261.4042,2022-04-05,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-22 16:00:00+07:00,660.5580,6.0000,72.0000,4.5000,13.5013,100.1354,173,6,468.3609,ISL056,473.4280,2023-06-22,16
2023-06-22 16:15:00+07:00,603.0061,6.0000,72.0000,4.5000,13.5013,100.1354,173,6,428.3692,ISL056,418.3324,2023-06-22,16
2023-06-22 16:30:00+07:00,543.1481,6.0000,60.0000,4.5000,13.5013,100.1354,173,6,399.2119,ISL056,367.2727,2023-06-22,16
